In [2]:
# Ensure that openai and python-dotenv are installed in the Jupyter Notebook environment
%pip install --upgrade openai python-dotenv

# Import necessary libraries
import json
import openai
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Load your existing dataset
with open('../data/mvp_dataset.json', 'r') as f:
    data = json.load(f)

# Convert to chat format
chat_data = []

for entry in data:
    user_message = None
    assistant_message = None
    for message in entry['messages']:
        if message['role'] == 'user':
            user_message = message['content']
        elif message['role'] == 'assistant':
            assistant_message = json.dumps(message['content'])  # Serialize the content to a string
    
    if user_message and assistant_message:
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": assistant_message}
        ]
        chat_data.append({"messages": messages})

# Save the new dataset in JSONL format
with open('../data/chat_dataset.jsonl', 'w') as f:
    for item in chat_data:
        f.write(json.dumps(item) + '\n')

print("Conversion complete! Chat-formatted data saved to 'data/chat_dataset.jsonl'")

# Upload the training file
with open("../data/chat_dataset.jsonl", "rb") as file:
    response = openai.File.create(
        file=file,
        purpose='fine-tune'
    )
training_file_id = response['id']
print(f"Training file ID: {training_file_id}")

# Verify the file upload
file_info = openai.File.retrieve(training_file_id)
print(file_info)

# Fine-tune the model
fine_tune_response = openai.FineTune.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo"  # Ensure you use the correct model ID
)
fine_tune_id = fine_tune_response['id']
print(f"Fine-tuning job ID: {fine_tune_id}")

# Check the status of the fine-tuning job
status = openai.FineTune.retrieve(fine_tune_id)
print(status)

# Optionally, monitor the fine-tuning process
def check_fine_tune_status(fine_tune_id):
    status = openai.FineTune.retrieve(fine_tune_id)
    status_str = status['status']
    print(f"Fine-tuning status: {status_str}")
    return status_str

# Monitor the fine-tuning job until it's done
while True:
    status_str = check_fine_tune_status(fine_tune_id)
    if status_str not in ['pending', 'running']:
        break
    time.sleep(60)  # Wait for 60 seconds before checking the status again

# Use the fine-tuned model once the fine-tuning is complete
if status_str == 'succeeded':
    response = openai.ChatCompletion.create(
        model=fine_tune_id,  # Use the ID of the fine-tuned model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Provide a lesson plan on Introduction to Addition and Subtraction suitable for students with Dyslexia and ADHD, for Kindergarten through Grade 2."}
        ],
        max_tokens=1000
    )

    print(response['choices'][0]['message']['content'])
else:
    print("Fine-tuning did not succeed.")


Note: you may need to restart the kernel to use updated packages.
Conversion complete! Chat-formatted data saved to 'data/chat_dataset.jsonl'


APIRemovedInV1: 

You tried to access openai.File, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
